# Expert F# - Chapter 17 - Language oriented programming
Nada Amin:
https://github.com/namin/spots/blob/master/probabilisticModeling/probabilisticModeling.fsx   
**Expert F# 4.0**   
Authors: Syme, Don, Granicz, Adam, Cisternino, Antonio   
http://www.apress.com/us/book/9781484207413   
**Stochastic Lambda Calculus and Monads of Probability Distribution**      
Norman Ramsey and Avi Pfeffer  
http://www.cs.tufts.edu/~nr/pubs/pmonad-abstract.html  
**Practical Probabilistic Programming with Monads**   
Adam Scibior, 
Zoubin Ghahramani, 
Andrew D. Gordon   
http://mlg.eng.cam.ac.uk/pub/pdf/SciGhaGor15.pdf   
** Probabilistic Programming: What It Is and How It Works** (in Scala)  
Noel Welsh   
https://www.youtube.com/watch?v=e1Ykk_CqKTY  
A Probabilistic Functional Programming Language DSL in Scala:   
https://github.com/noelwelsh/pfennig     

See also:    
https://www.google.ca/search?q=probability+monad   
https://people.cs.kuleuven.be/~tom.schrijvers/Research/talks/probability_monad.pdf  
http://jliszka.github.io/2013/08/12/a-frequentist-approach-to-probability.html  
https://www.google.ca/search?q=free+monad  

Nada Amin:   

> For my induction into Scala, I wanted to translate the probabilistic
monad of Chapter 9 of *[Expert F#][1]* (Introducing Language-Oriented
Programming). The idea, based on the paper *[Stochastic Lambda
Calculus and Monads of Probability Distributions][2]*, is to define a
probability monad to compute over distributions of a domain instead of
the domain itself. We limit ourselves to distributions over discrete
domains characterized by three functions:

> 1. **sampling**
2. **support** 
   (i.e. a set of values where all elements outside the set have zero
   chance of being sampled)
3. **expectation** of a function over the distribution 
   (e.g. the probability of selecting element `A` by evaluating the
   function `f(x) = 1` if `x` equals `A` and `0` otherwise)


[1]: http://www.expert-fsharp.com
[2]: http://www.cs.tufts.edu/~nr/pubs/pmonad-abstract.html

Syme et al, P487:  

## Example: Probabilistic Workflows
Workflows provide a fascinating way to embed a range of nontrivial, nonstandard computations into F#.
To give you a feel for this, this section defines a probabilistic workflow. That is, instead of writing
expressions to compute, say, integers, you instead write expressions that compute distributions of integers.
This case study is based on a paper by Ramsey and Pfeffer from 2002.

For the purposes of this section, you’re interested in distributions over discrete domains characterized
by three things:
* You want to be able to **sample** from a distribution (for example, sample an integer or
a coin flip).
* You want to compute the **support** of a distribution: that is, a set of values in which all
elements outside the set have zero chance of being sampled.
* You want to compute the **expectation** of a function over the distribution. For
example, you can compute the probability of selecting element A by evaluating the
expectation of the function `(fun x -> if x = A then 1.0 else 0.0)`.

In [1]:
open Microsoft.FSharp.Collections

In [2]:
type Distribution<'a when 'a : comparison> =
    abstract Sample : 'a
    abstract Support : Set<'a> // when 'a : comparison>
    abstract Expectation: ('a -> float) -> float

The simplest distribution is `always x`; this is a distribution that always samples to the same value. Its
expectation and support are easy to calculate.
The expectation of a function `H` is just `H` applied to the value,
and the support is a set containing the single value `x`.

In [3]:
let always x =
    { new Distribution<'a> with
         member d.Sample = x
         member d.Support = Set.singleton x
         member d.Expectation(H) = H(x) }

### Coinflip
The next distribution defined is `coinFlip`, which is a
distribution that models the ability to choose between two outcomes.

In [4]:
let rnd = System.Random()

let coinFlip (p:float) (d1:Distribution<'a>) (d2:Distribution<'a>) =
    if p < 0.0 || p > 1.0 then failwith "invalid probability in coinFlip"
    { new Distribution<'a> with
         member d.Sample =
             if rnd.NextDouble() < p then d1.Sample else d2.Sample
         member d.Support = d1.Support + d2.Support
         member d.Expectation(H) =
             p * d1.Expectation(H) + (1.0-p) * d2.Expectation(H) }

### A builder for probabilistic modeling
You can define a workflow builder for distribution objects.  

Note the all-important `bind` primitive; it combines two distributions, using the sample
from the first to guide the sample from the second. The support and expectation are calculated by taking
the support from the first and splaying it over the support of the second. The expectation is computed by
using the first distribution to compute the expectation of a function derived from the second. These are
standard results in probability theory, and they are the basic machinery you need to get going with some
interesting modeling.

In [6]:
let bind (dist:Distribution<'a>) (k: 'a -> Distribution<'b>) =
    { new Distribution<'b> with
         member d.Sample = (k(dist.Sample)).Sample
         member d.Support = Set.unionMany (dist.Support |> Set.map(fun d -> (k d).Support))
         member d.Expectation(H) = dist.Expectation(fun x -> (k x).Expectation(H)) }

In [22]:
type DistributionBuilder() =
    member x.Delay f = bind (always ()) f
    member x.Let(v,f) = bind (always v) f
    member x.Bind(d,f) = bind d f
    member x.Return v = always v
    member x.ReturnFrom vs = vs

Test it.

In [23]:
let dist = new DistributionBuilder()

In [24]:
let weightedCases (inp: ('a * float) list) =
    let rec coinFlips w l =
        match l with
        | []          -> failwith "no coinFlips"
        | [(d,_)]     -> always d
        | (d,p)::rest -> coinFlip (p/(1.0-w)) (always d) (coinFlips (w+p) rest)
    coinFlips 0.0 inp

let countedCases inp =
    let total = List.sumBy (fun (_,v) -> v) inp
    weightedCases (inp |> List.map (fun (x,v) -> (x,(float v/float total))))

type Outcome = Even | Odd | Zero
let roulette = countedCases [ Even,18; Odd,18; Zero,1]

In [25]:
roulette.Sample

Even

In [26]:
roulette.Expectation (function Even -> 10.0 | Odd -> 0.0 | Zero -> 0.0)

4.864864865

In [27]:
type Light =
    | Red
    | Green
    | Yellow

let trafficLightD = weightedCases [ Red,0.50; Yellow,0.10; Green, 0.40 ]

In [28]:
type Action = Stop | Drive

let cautiousDriver light =
    dist { match light with
           | Red -> return Stop
           | Yellow -> return! weightedCases [ Stop, 0.9; Drive, 0.1 ]
           | Green -> return Drive }

let aggressiveDriver light =
    dist { match light with
           | Red    -> return! weightedCases [ Stop, 0.9; Drive, 0.1 ]
           | Yellow -> return! weightedCases [ Stop, 0.1; Drive, 0.9 ]
           | Green  -> return Drive }

In [29]:
let otherLight light =
    match light with
    | Red -> Green
    | Yellow -> Red
    | Green -> Red
    
type CrashResult = Crash | NoCrash

let crash(driverOneD,driverTwoD,lightD) =
    dist { // Sample from the traffic light
           let! light = lightD

           // Sample the first driver's behavior given the traffic light
           let! driverOne = driverOneD light
           
           // Sample the second driver's behavior given the traffic light
           let! driverTwo = driverTwoD (otherLight light)

           // Work out the probability of a crash
           match driverOne, driverTwo with
             | Drive,Drive -> return! weightedCases [ Crash, 0.9; NoCrash, 0.1 ]
             | _ -> return NoCrash }

let model = crash(cautiousDriver,aggressiveDriver,trafficLightD)

let model2 = crash(aggressiveDriver,aggressiveDriver,trafficLightD)

In [31]:
model.Sample

NoCrash

In [32]:
model.Expectation(function Crash -> 1.0 | NoCrash -> 0.0)

0.0369

In [33]:
model2.Expectation(function Crash -> 1.0 | NoCrash -> 0.0)

0.0891